In [1]:
#Setup Notebook to load Django code
# From project root, run: jupyter-lab",

import os
import sys
import io
from pathlib import Path

django_project_dir = Path('../../..')
sys.path.insert(0, str(django_project_dir))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "traffic_stops.settings.dev")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django
django.setup()

In [2]:
import numpy as np
import pandas as pd
from django.db import connections
from django.db.models import Count, F, ExpressionWrapper, FloatField, Value, Q
from django.db.models.functions import ExtractYear, NullIf
from nc.models import ContrabandSummary      
from nc import query
import pprint

from pygments import highlight
from pygments.formatters import TerminalFormatter
from pygments.lexers import PostgresLexer
from sqlparse import format
from django.db.models import QuerySet


def print_sql(queryset: QuerySet):
    formatted = format(str(queryset.query), reindent=True)
    print(highlight(formatted, PostgresLexer(), TerminalFormatter()))

In [3]:
# ContrabandSummary.refresh()

## 1. CONTRABAND "HIT RATE"

In [65]:
group_by = ["driver_race"]
df = query.contraband_query(agency_id=80, group_by=group_by)
chart = query.hit_rate_chart(df, group_by=group_by)
table = query.hit_rate_table(df, group_by=group_by)
chart

# pprint.pprint(table, width=200, sort_dicts=False)

# Total across race for each yearb

{'Asian': [20.24],
 'Black': [27.66],
 'Hispanic': [17.13],
 'Native American': [23.33],
 'Other': [24.49],
 'White': [21.39]}

In [66]:
group_by = ["driver_race"]
df = query.contraband_query(agency_id=80, group_by=group_by)
chart = query.hit_rate_chart(df, group_by=group_by)
table = query.hit_rate_table(df, group_by=group_by)
chart

# pprint.pprint(table, width=200, sort_dicts=False)

# Total across race for each yearb

{'Asian': [20.24],
 'Black': [27.66],
 'Hispanic': [17.13],
 'Native American': [23.33],
 'Other': [24.49],
 'White': [21.39]}

## 2. CONTRABAND "HIT RATE" BY STOP PURPOSE

In [9]:
group_by = ["driver_race", "stop_purpose_group"]
df = query.contraband_query(agency_id=80, group_by=group_by)
chart = query.hit_rate_chart(df, group_by=group_by)
table = query.hit_rate_table(df, group_by=group_by)
# pprint.pprint(chart, sort_dicts=False)

# pprint.pprint(table, width=200, sort_dicts=False)

# response = {"datasets": {
#     "Regulatory and Equipment": [
#         {'label': 'Asian', 'data': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]},
#     ],
#     "Safety Violation": {},
#     "Other": {},
# }}



In [50]:
df1 = df.pivot_table(
        index="year", columns=["stop_purpose_group", "driver_race", ], values="contraband_found_count", fill_value=0
    ).astype("Int64")
df1.columns.get_level_values(0)

Index(['Other', 'Other', 'Other', 'Other', 'Other', 'Other',
       'Regulatory and Equipment', 'Regulatory and Equipment',
       'Regulatory and Equipment', 'Regulatory and Equipment',
       'Regulatory and Equipment', 'Regulatory and Equipment',
       'Safety Violation', 'Safety Violation', 'Safety Violation',
       'Safety Violation', 'Safety Violation', 'Safety Violation'],
      dtype='object', name='stop_purpose_group')

In [52]:
df1 = df.pivot_table(
        index=["stop_purpose_group", "year", ], columns=["driver_race"], values="contraband_found_count", fill_value=0
    ).astype("Int64")
df1

driver_race              Asian  Black  Hispanic  Native American  Other  White
stop_purpose_group year                                                       
Other              2002      0     32         4                0      1      8
                   2003      0     17         2                0      0      3
                   2004      0     16         0                0      0      4
                   2005      0      9         2                0      0      3
                   2006      0     22         0                0      0      7
...                        ...    ...       ...              ...    ...    ...
Safety Violation   2019      0     92         3                0      0     10
                   2020      0     59         3                1      0      3
                   2021      0     79         8                1      0      8
                   2022      0     93        13                0      0      2
                   2023      1     58        16                0      0      6

[66 rows x 6 columns]

In [53]:
df1 = df.pivot_table(
        index=["stop_purpose_group", "year", ], columns=["driver_race"], values="contraband_found_count", fill_value=0
    ).astype("Int64")
df2 = df1.reset_index()
df3 = df2[df2["stop_purpose_group"] == "Other"]
df3.to_dict("records")
None

## 3. CONTRABAND "HIT RATE" BY STOP PURPOSE

In [57]:
group_by = ["stop_purpose_group", "driver_race", "contraband_type"]
df = query.contraband_query(agency_id=80, group_by=group_by)
chart = query.hit_rate_chart(df, group_by=group_by)
table = query.hit_rate_table(df, group_by=group_by)
df

,stop_purpose_group,driver_race,contraband_type,year,search_count,contraband_found_count
0,Other,Asian,Alcohol,2020,1,0
1,Other,Asian,Drugs,2020,1,0
2,Other,Asian,Money,2020,1,0
3,Other,Asian,Other,2020,1,1
4,Other,Asian,Weapons,2020,1,0
...,...,...,...,...,...,...
1405,Safety Violation,White,None,2019,15,0
1406,Safety Violation,White,None,2020,10,0
1407,Safety Violation,White,None,2021,8,0
1408,Safety Violation,White,None,2022,11,0


## 4. Contraband "Hit rate" by type

In [7]:
group_by = ["contraband_type"]
df = query.contraband_query(agency_id=80, group_by=group_by)
chart = query.hit_rate_chart(df, group_by=group_by)
table = query.hit_rate_table(df, group_by=group_by)
chart

{'Alcohol': [7.14],
 'Drugs': [63.04],
 'Money': [10.16],
 'Other': [5.78],
 'Weapons': [15.71]}

{'Alcohol': [7.14],
 'Drugs': [63.04],
 'Money': [10.16],
 'Other': [5.78],
 'Weapons': [15.71]}

In [128]:
table = df.pivot_table(index="year", columns=["driver_race"], values="contraband_found_count", fill_value=0).astype("Int64")
table = {"labels": list(table.index), "datasets": []}
for dataset in table.to_dict("list"):
    

{'Asian': [0,
  0,
  0,
  0,
  0,
  2,
  4,
  1,
  0,
  1,
  0,
  0,
  2,
  0,
  1,
  0,
  0,
  2,
  1,
  1,
  1,
  1,
  1],
 'Black': [0,
  224,
  107,
  95,
  86,
  150,
  259,
  415,
  148,
  319,
  392,
  343,
  303,
  294,
  388,
  227,
  150,
  203,
  349,
  254,
  344,
  572,
  352],
 'Hispanic': [0,
  43,
  17,
  11,
  13,
  17,
  31,
  38,
  12,
  22,
  25,
  28,
  22,
  25,
  20,
  18,
  14,
  28,
  25,
  18,
  25,
  51,
  42],
 'Native American': [0,
  0,
  0,
  1,
  0,
  0,
  1,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  0],
 'Other': [0,
  3,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  1,
  0,
  1],
 'White': [0,
  43,
  29,
  25,
  14,
  30,
  54,
  57,
  28,
  47,
  41,
  40,
  35,
  32,
  34,
  26,
  21,
  16,
  30,
  24,
  25,
  20,
  14]}

In [53]:
df1 = df.groupby(['driver_race'])[['search_count', 'contraband_found_count']].agg('sum').reset_index()
df1["hit_rate"] = df1.contraband_found_count / df1.search_count * 100
df1

,driver_race,search_count,contraband_found_count,hit_rate
0,Asian,84,18,21.428571
1,Black,18253,5974,32.728866
2,Hispanic,2569,545,21.214480
3,Native American,30,9,30.000000
4,Other,49,12,24.489796
5,White,2492,685,27.487961


In [59]:
table = df1.pivot_table(columns="driver_race", values=["hit_rate"])
table.to_dict("list")

{'Asian': [21.428571428571427],
 'Black': [32.72886648770065],
 'Hispanic': [21.214480342545738],
 'Native American': [30.0],
 'Other': [24.489795918367346],
 'White': [27.48796147672552]}

In [27]:
%%time

qs = (
    
    .annotate(year=ExtractYear("date"))
    .values("year", "driver_race_comb", "stop_purpose_group", "contraband_type")
    .annotate(
        search_count=Count("search_id", distinct=True),
        contraband_found_count=Count("contraband_id", distinct=True),
    )
    .annotate(
        hit_rate=ExpressionWrapper(
            F("contraband_found_count") * 1.0 / NullIf('search_count', Value(0)), output_field=FloatField()
        )
    )
    .order_by("year")
)
print(qs.explain(analyze=True, verbose=True))
pd.DataFrame(qs)

GroupAggregate  (cost=388782.24..409052.15 rows=413639 width=87) (actual time=391.816..487.000 rows=1528 loops=1)
  Output: driver_race, stop_purpose_group, contraband_type, (date_part('year'::text, stop_date)), count(DISTINCT search_id), count(DISTINCT contraband_id), (((count(DISTINCT contraband_id))::numeric * 1.0) / (NULLIF(count(DISTINCT search_id), 0))::numeric)
  Group Key: (date_part('year'::text, nc_contrabandsummary.stop_date)), nc_contrabandsummary.driver_race, nc_contrabandsummary.stop_purpose_group, nc_contrabandsummary.contraband_type
  ->  Sort  (cost=388782.24..389885.45 rows=441286 width=47) (actual time=391.724..425.388 rows=399617 loops=1)
        Output: driver_race, stop_purpose_group, contraband_type, (date_part('year'::text, stop_date)), search_id, contraband_id
        Sort Key: (date_part('year'::text, nc_contrabandsummary.stop_date)), nc_contrabandsummary.driver_race, nc_contrabandsummary.stop_purpose_group, nc_contrabandsummary.contraband_type
        Sort Me

,driver_race_comb,stop_purpose_group,contraband_type,year,search_count,contraband_found_count,hit_rate
0,Black,Other,None,2001,0,0,NaN
1,Asian,Other,None,2002,1,0,0.0
2,Asian,Regulatory and Equipment,None,2002,4,0,0.0
3,Asian,Safety Violation,None,2002,3,0,0.0
4,Black,Other,Alcohol,2002,34,34,1.0
...,...,...,...,...,...,...,...
1523,White,Safety Violation,Drugs,2023,8,8,1.0
1524,White,Safety Violation,Money,2023,8,8,1.0
1525,White,Safety Violation,Other,2023,8,8,1.0
1526,White,Safety Violation,Weapons,2023,8,8,1.0


In [26]:
print_sql(qs)

SELECT "nc_contrabandsummary"."driver_race",
       "nc_contrabandsummary"."stop_purpose_group",
       "nc_contrabandsummary"."contraband_type",
       EXTRACT('year'
               FROM "nc_contrabandsummary"."stop_date") AS "year",
       COUNT(DISTINCT "nc_contrabandsummary"."search_id") AS "search_count",
       COUNT(DISTINCT "nc_contrabandsummary"."contraband_id") AS "contraband_found_count",
       ((COUNT(DISTINCT "nc_contrabandsummary"."contraband_id") * 1.0) / NULLIF(COUNT(DISTINCT "nc_contrabandsummary"."search_id"), 0)) AS "hit_rate"
FROM "nc_contrabandsummary"
WHERE "nc_contrabandsummary"."agency_id" = 80
GROUP BY "nc_contrabandsummary"."driver_race",
         "nc_contrabandsummary"."stop_purpose_group",
         "nc_contrabandsummary"."contraband_type",
         EXTRACT('year'
                 FROM "nc_contrabandsummary"."stop_date")
ORDER BY "year" ASC



In [155]:
from enum import Enum

class StopPurpose(Enum):
    SPEED_LIMIT_VIOLATION         = 1   # Safety Violation
    STOP_LIGHT_SIGN_VIOLATION     = 2   # Safety Violation
    DRIVING_WHILE_IMPAIRED        = 3   # Safety Violation
    SAFE_MOVEMENT_VIOLATION       = 4   # Safety Violation
    VEHICLE_EQUIPMENT_VIOLATION   = 5   # Regulatory and Equipment
    VEHICLE_REGULATORY_VIOLATION  = 6   # Regulatory and Equipment
    OTHER_MOTOR_VEHICLE_VIOLATION = 9   # Regulatory and Equipment
    SEAT_BELT_VIOLATION           = 7   # Regulatory and Equipment
    INVESTIGATION                 = 8   # Investigatory
    CHECKPOINT                    = 10  # Investigatory
    
    @classmethod
    def safety_violation(cls):
        return [cls.SPEED_LIMIT_VIOLATION.value, cls.STOP_LIGHT_SIGN_VIOLATION.value, cls.DRIVING_WHILE_IMPAIRED.value, cls.SAFE_MOVEMENT_VIOLATION.value]
    
    @classmethod
    def regulatory_equipment(cls):
        return [cls.VEHICLE_EQUIPMENT_VIOLATION.value, cls.VEHICLE_REGULATORY_VIOLATION.value, cls.OTHER_MOTOR_VEHICLE_VIOLATION.value, cls.SEAT_BELT_VIOLATION.value]
    
    @classmethod
    def investigatory(cls):
        return [cls.INVESTIGATION.value, cls.CHECKPOINT.value]

contraband_summary_sql = f"""
    WITH 
        contraband_groups AS (
            SELECT
                *
                , (CASE WHEN nc_contraband.pints > 0 OR nc_contraband.gallons > 0 THEN true
                        ELSE false
                   END) AS alcohol_found
                , (CASE WHEN nc_contraband.ounces > 0 OR nc_contraband.pounds > 0 OR nc_contraband.dosages > 0 OR nc_contraband.grams > 0 OR nc_contraband.kilos > 0 THEN true
                        ELSE false
                   END) AS drugs_found
                , (CASE WHEN nc_contraband.money > 0 THEN true
                        ELSE false
                   END) AS money_found
                , (CASE WHEN nc_contraband.dollar_amount > 0 THEN true
                        ELSE false
                   END) AS other_found
                , (CASE WHEN nc_contraband.weapons > 0 THEN true
                        ELSE false
                   END) AS weapons_found
            FROM nc_contraband
        ),
        contraband_types_without_id AS (
            SELECT
                contraband_id
                , person_id
                , search_id
                , stop_id
                , unnest(ARRAY['Alcohol', 'Drugs', 'Money', 'Other', 'Weapons']) AS contraband_type
                , unnest(ARRAY[alcohol_found, drugs_found, money_found, other_found, weapons_found]) AS contraband_found
            FROM contraband_groups
        ), 
        contraband_types AS (
            SELECT
                ROW_NUMBER() OVER () AS contraband_type_id
                , *
            FROM contraband_types_without_id
        ),
        contraband_summary AS (
            SELECT
                nc_stop.stop_id
                , date AT TIME ZONE 'America/New_York' AS stop_date
                , nc_stop.agency_id
                , (CASE WHEN nc_stop.purpose IN ({",".join(map(str, StopPurpose.safety_violation()))}) THEN 'Safety Violation'
                        WHEN nc_stop.purpose IN ({",".join(map(str, StopPurpose.investigatory()))}) THEN 'Investigatory'
                        WHEN nc_stop.purpose IN ({",".join(map(str, StopPurpose.regulatory_equipment()))}) THEN 'Regulatory and Equipment'
                        ELSE 'Other'
                   END) as stop_purpose_group
                , (CASE WHEN nc_person.ethnicity = 'H' THEN 'Hispanic'
                        WHEN nc_person.ethnicity = 'N' AND nc_person.race = 'A' THEN 'Asian'
                        WHEN nc_person.ethnicity = 'N' AND nc_person.race = 'B' THEN 'Black'
                        WHEN nc_person.ethnicity = 'N' AND nc_person.race = 'I' THEN 'Native American'
                        WHEN nc_person.ethnicity = 'N' AND nc_person.race = 'U' THEN 'Other'
                        WHEN nc_person.ethnicity = 'N' AND nc_person.race = 'W' THEN 'White'
                   END) as driver_race
                , (CASE WHEN nc_person.gender = 'M' THEN 'Male'
                        WHEN nc_person.gender = 'F' THEN 'Female'
                   END) as driver_gender
                , (nc_search.search_id IS NOT NULL) AS driver_searched
                , nc_search.search_id
                , (contraband_id IS NOT NULL) AS contraband_id_found
                , contraband_found
                , contraband_id
                , contraband_type_id
                , contraband_type AS contraband_type_found
            FROM "nc_stop"
            INNER JOIN "nc_person"
                ON ("nc_stop"."stop_id" = "nc_person"."stop_id" AND "nc_person"."type" = 'D')
            LEFT OUTER JOIN "nc_search"
                ON ("nc_stop"."stop_id" = "nc_search"."stop_id")
            LEFT OUTER JOIN "contraband_types"
                ON ("nc_stop"."stop_id" = "contraband_types"."stop_id")
        )
"""

In [163]:
def contraband_hit_rate_by_type():
    df = pd.read_sql(
        f"""
        {contraband_summary_sql}
        SELECT
            contraband_type_found
            , count(stop_id) AS stop_count
            , count(search_id) FILTER (WHERE driver_searched = true) AS search_count
            , count(contraband_type_id) FILTER (WHERE contraband_found = true) AS countraband_found_count
        FROM contraband_summary
        WHERE agency_id IN (80)
          AND driver_searched = true
          AND contraband_type_found <> 'None'
        GROUP BY 1
        ORDER BY 1
        """,
        connections['traffic_stops_nc'],
    )
    df["contraband_hit_rate"] = df.countraband_found_count / df.search_count
    return df
contraband_hit_rate_by_type()


/var/folders/xj/05km0jzj6zv0nnrw6x4f4j8h0000gn/T/ipykernel_37974/562806849.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


,contraband_type_found,stop_count,search_count,countraband_found_count,contraband_hit_rate
0,Alcohol,7243,7243,517,0.071379
1,Drugs,7243,7243,4566,0.630402
2,Money,7243,7243,736,0.101615
3,Other,7243,7243,419,0.057849
4,Weapons,7243,7243,1138,0.157117
